# Fitting 2d Toy Datasets with the FKLD

In [ ]:
# Import packages
import numpy as np
import torch
import normflows as nf
import larsflow as lf

from matplotlib import pyplot as plt

from tqdm import tqdm

In [ ]:
# Get device
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [ ]:
# Function for model creation

def create_model(p, base='gauss'):
    # Set up model

    # Define flows
    K = 8
    torch.manual_seed(10)

    latent_size = 2
    b = torch.Tensor([1 if i % 2 == 0 else 0 for i in range(latent_size)])
    flows = []
    for i in range(K):
        param_map = nf.nets.MLP([latent_size // 2, 32, 32, latent_size], init_zeros=True)
        flows += [nf.flows.AffineCouplingBlock(param_map)]
        flows += [nf.flows.Permute(latent_size, mode='swap')]
        flows += [nf.flows.ActNorm(latent_size)]

    # Set prior and q0
    if base == 'resampled':
        a = nf.nets.MLP([latent_size, 256, 256, 1], output_fn="sigmoid")
        q0 = lf.distributions.ResampledGaussian(latent_size, a, 100, 0.1, trainable=False)
    elif base == 'gaussian_mixture':
        n_modes = 10
        q0 = nf.distributions.GaussianMixture(n_modes, latent_size, trainable=True,
                                              loc=(np.random.rand(n_modes, latent_size) - 0.5) * 5,
                                              scale=0.5 * np.ones((n_modes, latent_size)))
    elif base == 'gauss':
        q0 = nf.distributions.DiagGaussian(latent_size, trainable=False)
    else:
        raise NotImplementedError('This base distribution is not implemented.')

    # Construct flow model
    model = lf.NormalizingFlow(q0=q0, flows=flows, p=p)

    # Move model on GPU if available
    return model.to(device)


# Function to train model

def train(model, max_iter=20000, num_samples=2 ** 10, lr=1e-3, weight_decay=1e-3, 
          q0_weight_decay=1e-4):
    # Do mixed precision training
    optimizer = torch.optim.Adam(model.parameters(),  lr=lr, weight_decay=weight_decay)
    model.train()

    for it in tqdm(range(max_iter)):
        
        x = model.p.sample(num_samples)

        loss = model.forward_kld(x)

        loss.backward()
        optimizer.step()

        # Clear gradients
        nf.utils.clear_grad(model)

In [ ]:
# Plot function
def plot_results(model, target=True, a=False, save=False, prefix=''):
    # Prepare z grid for evaluation
    grid_size = 300
    xx, yy = torch.meshgrid(torch.linspace(-3, 3, grid_size), torch.linspace(-3, 3, grid_size))
    zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
    zz = zz.to(device)
    
    log_prob = model.p.log_prob(zz).to('cpu').view(*xx.shape)
    prob = torch.exp(log_prob)
    prob[torch.isnan(prob)] = 0
    prob_target = prob.data.numpy()
    
    if target:
        plt.figure(figsize=(15, 15))
        plt.pcolormesh(xx, yy, prob_target)
        #cs = plt.contour(xx, yy, prob_target, [.025, .15, .7], colors='w', linewidths=3)#, linestyles='dashed')
        #for c in cs.collections:
        #    c.set_dashes([(0, (10.0, 10.0))])
        plt.gca().set_aspect('equal', 'box')
        plt.axis('off')
        if save:
            plt.savefig(prefix + 'target.png', dpi=300)
        plt.show()

    model.eval()
    log_prob = model.log_prob(zz).to('cpu').view(*xx.shape)

    prob = torch.exp(log_prob.to('cpu').view(*xx.shape))
    prob[torch.isnan(prob)] = 0
    prob_model = prob.data.numpy()

    plt.figure(figsize=(15, 15))
    plt.pcolormesh(xx, yy, prob_model)#, cmap=plt.get_cmap('coolwarm'))
    #cs = plt.contour(xx, yy, prob_model, [.025, .2, .35], colors='w', linewidths=3)#, linestyles='dashed')
    #for c in cs.collections:
    #    c.set_dashes([(0, (10.0, 10.0))])
    plt.gca().set_aspect('equal')
    plt.axis('off')
    if save:
        plt.savefig(prefix + 'model.png', dpi=300)
    plt.show()

    log_prob = model.q0.log_prob(zz).to('cpu').view(*xx.shape)
    prob = torch.exp(log_prob)
    prob[torch.isnan(prob)] = 0
    prob_base = prob.data.numpy()

    plt.figure(figsize=(15, 15))
    plt.pcolormesh(xx, yy, prob_base)
    #cs = plt.contour(xx, yy, prob_base, [.025, .075, .135], colors='w', linewidths=3)#, linestyles='dashed')
    #for c in cs.collections:
    #    c.set_dashes([(0, (10.0, 10.0))])
    plt.gca().set_aspect('equal')
    plt.axis('off')
    if save:
        plt.savefig(prefix + 'base.png', dpi=300)
    plt.show()
    
    if a:
        prob = model.q0.a(zz).to('cpu').view(*xx.shape)
        prob[torch.isnan(prob)] = 0

        plt.figure(figsize=(15, 15))
        plt.pcolormesh(xx, yy, prob.data.numpy())
        plt.gca().set_aspect('equal')
        plt.axis('off')
        if save:
            plt.savefig(prefix + 'a.png', dpi=300)
        plt.show()
    
    # Compute KLD
    eps = 1e-10
    kld = np.sum(prob_target * np.log((prob_target + eps) / (prob_model + eps)) * 6 ** 2 / grid_size ** 2)
    print('KL divergence: %f' % kld)

In [ ]:
# Train models
p = [nf.distributions.TwoMoons(), nf.distributions.CircularGaussianMixture(), nf.distributions.RingMixture()]
name = ['moons', 'circle', 'rings']

for i in range(len(p)):
    # Train model with Gaussain base distribution
    model = create_model(p[i], 'gauss')
    train(model)
    # Plot and save results
    plot_results(model, save=False,
                 prefix='results/2d_distributions/fkld/rnvp/' 
                 + name[i] + '_gauss_')
    
    # Train model with mixture of Gaussians base distribution
    model = create_model(p[i], 'gaussian_mixture')
    train(model)
    # Plot and save results
    plot_results(model, save=True,
                 prefix='results/2d_distributions/fkld/rnvp/' 
                 + name[i] + '_gaussian_mixture_')
    
    # Train model with resampled base distribution
    model = create_model(p[i], 'resampled')
    train(model)
    # Plot and save results
    plot_results(model, save=True, a=True,
                 prefix='results/2d_distributions/fkld/rnvp/' 
                 + name[i] + '_resampled_')